In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = pd.DataFrame(df["loan_status"].copy(), columns=(["loan_status"]))

In [7]:
# One-Hot Encoding Categorical Variables here
X2 = pd.get_dummies(X)
X2.shape

(68817, 95)

# Split the Data into Training and Testing

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, 
                                                    y, 
                                                    random_state=7777, 
                                                    stratify=y)
X_train.shape

(51612, 95)

In [9]:
# Scaling numerical columns

from sklearn.preprocessing import StandardScaler
X_scaler_train = StandardScaler().fit(X_train)
X_scaler_test = StandardScaler().fit(X_test)

X_train_scaled = X_scaler_train.transform(X_train)
X_test_scaled = X_scaler_test.transform(X_test)

In [13]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

In [17]:
clf = BalancedRandomForestClassifier(n_estimators=100,  random_state=1)
clf.fit(X_train_scaled, y_train) 

BalancedRandomForestClassifier(random_state=1)

In [19]:
# Calculated the balanced accuracy score
y_pred = clf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7538837924185471

In [21]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   54,    33],
       [ 1933, 15185]], dtype=int64)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.62      0.89      0.05      0.74      0.54        87
   low_risk       1.00      0.89      0.62      0.94      0.74      0.57     17118

avg / total       0.99      0.89      0.62      0.93      0.74      0.57     17205



In [25]:
# List the features sorted in descending order by feature importance
fea_imp=clf.feature_importances_
featNames=X_test.columns
sorted(zip(fea_imp, featNames), reverse=True)

[(0.08992704982209049, 'total_rec_prncp'),
 (0.06122322067419792, 'total_pymnt'),
 (0.05803548009211756, 'last_pymnt_amnt'),
 (0.04716737680565753, 'total_rec_int'),
 (0.04628578927067361, 'total_pymnt_inv'),
 (0.03442989191377501, 'int_rate'),
 (0.021912880008775417, 'issue_d_Jan-2019'),
 (0.016949205877611558, 'dti'),
 (0.01605941842053027, 'mths_since_recent_inq'),
 (0.015521708459630256, 'installment'),
 (0.015347334893409982, 'max_bal_bc'),
 (0.015301081795348191, 'annual_inc'),
 (0.014920407480824934, 'out_prncp'),
 (0.014798834297886974, 'mo_sin_old_il_acct'),
 (0.014762452185771205, 'tot_cur_bal'),
 (0.014739144127810353, 'avg_cur_bal'),
 (0.01471465849898004, 'revol_bal'),
 (0.014584939367434012, 'il_util'),
 (0.014380510460056927, 'mo_sin_old_rev_tl_op'),
 (0.014348960501327163, 'total_bal_il'),
 (0.014151375187734974, 'tot_hi_cred_lim'),
 (0.013953169668322685, 'bc_open_to_buy'),
 (0.013900687062292423, 'all_util'),
 (0.013316246048342847, 'total_bc_limit'),
 (0.013263505248

### Easy Ensemble AdaBoost Classifier

In [27]:
# Train the EasyEnsembleClassifier
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=1)
clf.fit(X_train_scaled, y_train)

AdaBoostClassifier(n_estimators=100, random_state=1)

In [28]:
# Calculated the balanced accuracy score
y_pred = clf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.666403785488959

In [29]:
confusion_matrix(y_test, y_pred)

array([[   29,    58],
       [    9, 17109]], dtype=int64)

In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.76      0.33      1.00      0.46      0.58      0.31        87
   low_risk       1.00      1.00      0.33      1.00      0.58      0.36     17118

avg / total       1.00      1.00      0.34      1.00      0.58      0.36     17205

